In [1]:
library(infercnv)
library(Matrix)
library(Seurat)
library(SeuratDisk)
library(tidyverse)
library(stringr)

Registered S3 method overwritten by 'spatstat':
  method     from
  print.boxx cli 

Attaching SeuratObject

Registered S3 method overwritten by 'SeuratDisk':
  method            from  
  as.sparse.H5Group Seurat

── Attaching packages ───────────────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.3     ✔ purrr   0.3.4
✔ tibble  3.1.0     ✔ dplyr   1.0.8
✔ tidyr   1.1.3     ✔ stringr 1.4.0
✔ readr   2.0.1     ✔ forcats 0.5.1

── Conflicts ──────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ tidyr::expand() masks Matrix::expand()
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
✖ tidyr::pack()   masks Matrix::pack()
✖ tidyr::unpack() masks Matrix::unpack()



In [2]:
Convert('/data/countMatrix/rawSingletForR15Kcells.h5ad', assay = 'RNA' ,dest = "h5seurat", overwrite = TRUE) # Singlet H5ad - change to your own directory

Warning message:
“Unknown file type: h5ad”
Creating h5Seurat file for version 3.1.5.9900

Adding X as data

Adding X as counts

Adding meta.features from var



In [3]:
dat <- LoadH5Seurat('/data/countMatrix/rawSingletForR15Kcells.h5seurat')

Validating h5Seurat file

Initializing RNA with data

Adding counts for RNA

Adding feature-level metadata for RNA

Adding command information

Adding cell-level metadata

Adding miscellaneous information

Adding tool-specific results



In [4]:
# dat = RenameCells(dat, Idents(dat))
dat <- SetIdent(dat, value = "Barcode")


# inferCNV - ALL samples

In [5]:
sampleMeta = dat[[c('Barcode', 'guides')]]

In [6]:
write.table(
    sampleMeta, 
    file = "/data/annotations.txt", 
    sep = "\t", 
    row.names = FALSE, 
    col.names = FALSE, 
    quote = FALSE)

In [7]:
annotations = read.table('/data/annotations.txt', sep = '\t')
head(annotations)

,V1,V2
,<chr>,<chr>
1,AAACCCAAGCTAATGA-1-0,CTTCAAGAGCAACAGTGCTG_+
2,AAACCCAAGTATGTAG-1-0,GACACCTTCTTCCCCAGCCC_+
3,AAACCCACAAGTCATC-1-0,GACACCTTCTTCCCCAGCCC_+
4,AAACCCACACACACTA-1-0,TTCTTCCCCAGCCCAGGTAA_+
5,AAACCCACACCCATAA-1-0,GACACCTTCTTCCCCAGCCC_+
6,AAACCCACACGCACCA-1-0,GACACCTTCTTCCCCAGCCC_+


In [8]:
dat <- SetIdent(dat, value = "Barcode")
counts = GetAssayData(object = dat, slot = "counts")
colnames(counts) <- Idents(object = dat)

In [9]:
infercnv_obj <- CreateInfercnvObject(
raw_counts_matrix=counts,
annotations_file='/data/annotations.txt',
delim="\t",
gene_order_file='/data/inferCNVgeneName.txt',
ref_group_names = c('NT')
)

INFO [2022-02-17 12:29:53] Parsing gene order file: /data/inferCNVgeneName.txt
INFO [2022-02-17 12:29:53] Parsing cell annotations file: /data/annotations.txt
INFO [2022-02-17 12:29:53] ::order_reduce:Start.
INFO [2022-02-17 12:29:55] .order_reduce(): expr and order match.
INFO [2022-02-17 12:29:56] ::process_data:order_reduce:Reduction from positional data, new dimensions (r,c) = 36601,33461 Total=356413122 Min=0 Max=2617.
INFO [2022-02-17 12:29:57] num genes removed taking into account provided gene ordering list: 1280 = 3.49717220840961% removed.
INFO [2022-02-17 12:29:57] -filtering out cells < 100 or > Inf, removing 0 % of cells
INFO [2022-02-17 12:30:05] validating infercnv_obj


In [10]:
infercnv_obj = infercnv::run(
    infercnv_obj,
    cutoff=0.1, # cutoff=1 works well for Smart-seq2, and cutoff=0.1 works well for 10x Genomics
    out_dir='/data/inferCNVResults15Kcells/', 
    cluster_by_groups=TRUE, 
    denoise=TRUE,
    HMM=TRUE,
    num_threads=24,
)


INFO [2022-02-17 12:30:13] ::process_data:Start
INFO [2022-02-17 12:30:13] Creating output path /data/inferCNVResults15Kcells/
INFO [2022-02-17 12:30:13] Checking for saved results.
INFO [2022-02-17 12:30:13] 

	STEP 1: incoming data

INFO [2022-02-17 12:31:52] 

	STEP 02: Removing lowly expressed genes

INFO [2022-02-17 12:31:52] ::above_min_mean_expr_cutoff:Start
INFO [2022-02-17 12:31:56] Removing 28149 genes from matrix as below mean expr threshold: 0.1
INFO [2022-02-17 12:32:00] validating infercnv_obj
INFO [2022-02-17 12:32:00] There are 7172 genes and 33461 cells remaining in the expr matrix.
INFO [2022-02-17 12:32:17] no genes removed due to min cells/gene filter
INFO [2022-02-17 12:33:58] 

	STEP 03: normalization by sequencing depth

INFO [2022-02-17 12:33:58] normalizing counts matrix by depth
INFO [2022-02-17 12:34:20] Computed total sum normalization factor as median libsize: 7642.000000
INFO [2022-02-17 12:34:20] Adding h-spike
INFO [2022-02-17 12:34:20] -hspike modeling 

In [12]:
head(infercnv_obj@expr.data)


,AAACCCAAGCTAATGA-1-0,AAACCCAAGTATGTAG-1-0,AAACCCACAAGTCATC-1-0,AAACCCACACACACTA-1-0,AAACCCACACCCATAA-1-0,AAACCCACACGCACCA-1-0,AAACCCACACTGCGAC-1-0,AAACCCACAGGACTTT-1-0,AAACCCAGTTGGGTTT-1-0,AAACCCATCAGGTAAA-1-0,⋯,TTTGTTGCAATCCTTT-1-2,TTTGTTGCATGACTAC-1-2,TTTGTTGGTCAAGTTC-1-2,TTTGTTGGTGGATTTC-1-2,TTTGTTGGTTACAGCT-1-2,TTTGTTGGTTCTTGTT-1-2,TTTGTTGGTTGCCGAC-1-2,TTTGTTGTCCGGTAGC-1-2,TTTGTTGTCTAGAACC-1-2,TTTGTTGTCTTCTGGC-1-2
NOC2L,1.0012,0.8496507,1.0012,1.0012,1.0012,1.0012,1.0012,1.0012,1.0012,1.0012,⋯,1.0012,1.0012,1.0012,1.0012,0.8995852,0.8588233,1.0012,1.0012,1.0012,1.0012
ISG15,1.0012,0.8509542,1.0012,1.0012,1.0012,1.0012,1.0012,1.0012,1.0012,1.0012,⋯,1.0012,1.0012,1.0012,1.0012,0.8996055,0.8595151,1.0012,1.0012,1.0012,1.0012
TNFRSF4,1.0012,0.8522806,1.0012,1.0012,1.0012,1.0012,1.0012,1.0012,1.0012,1.0012,⋯,1.0012,1.0012,1.0012,1.0012,0.8997827,0.8602901,1.0012,1.0012,1.0012,1.0012
SDF4,1.0012,0.8534545,1.0012,1.0012,1.0012,1.0012,1.0012,1.0012,1.0012,1.0012,⋯,1.0012,1.0012,1.0012,1.0012,0.8999158,0.8609641,1.0012,1.0012,1.0012,1.0012
B3GALT6,1.0012,0.8546630,1.0012,1.0012,1.0012,1.0012,1.0012,1.0012,1.0012,1.0012,⋯,1.0012,1.0012,1.0012,1.0012,0.9001891,0.8617201,1.0012,1.0012,1.0012,1.0012
UBE2J2,1.0012,0.8557378,1.0012,1.0012,1.0012,1.0012,1.0012,1.0012,1.0012,1.0012,⋯,1.0012,1.0012,1.0012,1.0012,0.9004141,0.8623833,1.0012,1.0012,1.0012,1.0012


In [13]:
cnvRes <- infercnv_obj@expr.data
# write results
write.table(
cnvRes,
file = stringr::str_interp('/data/inferCNVResults15Kcells/cnvResAllExpr.txt'),
sep = '\t',
row.names = T,
col.names = T,
quote = F)